In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image

In [8]:
if SparkSession.builder.getOrCreate().sparkContext:
    SparkSession.builder.getOrCreate().sparkContext.stop()
    
spark = SparkSession.builder \
    .appName("PCA Example") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.maxResultSize", "4g") \
    .getOrCreate()


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Applications/anaconda3/envs/test-spark/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: SparkSession$ does not exist in the JVM

In [3]:
import glob

image_folder = './data/Test1/Apple Braeburn/'
image_files = glob.glob(image_folder + '*.jpg')

def load_image_as_vector(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image_resized = cv2.resize(image, (64, 64))  # Redimensionner pour la simplicité
    image_vector = image_resized.flatten().tolist()  # Aplatir en un seul vecteur
    return (Vectors.dense(image_vector),)

# Charger les images et créer un DataFrame Spark
image_vectors = [load_image_as_vector(image_file) for image_file in image_files]
df = spark.createDataFrame(image_vectors, ["features"])


In [6]:
df = df.repartition(200)  # Répartir les données en 10 partitions

In [7]:
pca = PCA(k=20, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(df)
df_pca = pca_model.transform(df)

24/05/27 23:33:41 WARN TaskSetManager: Stage 13 contains a task of very large size (2204 KiB). The maximum recommended task size is 1000 KiB.


[319,934s][warning][gc,alloc] Executor task launch worker for task 5.0 in stage 25.0 (TID 464): Retried waiting for GCLocker too often allocating 8390658 words


24/05/27 23:35:11 ERROR Executor: Exception in task 5.0 in stage 25.0 (TID 464)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2098)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2228)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1687)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:489)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:447)
	at org.apache.spark.serializer.JavaDeserializationStream.readObject(JavaSerializer.scala:87)
	at org.apache.spark.serializer.DeserializationStream.readV

[320,263s][warning][gc,alloc] Executor task launch worker for task 8.0 in stage 25.0 (TID 467): Retried waiting for GCLocker too often allocating 8390658 words


24/05/27 23:35:11 ERROR Executor: Exception in task 8.0 in stage 25.0 (TID 467)
java.lang.OutOfMemoryError: Java heap space
24/05/27 23:35:11 ERROR SparkUncaughtExceptionHandler: [Container in shutdown] Uncaught exception in thread Thread[Executor task launch worker for task 8.0 in stage 25.0 (TID 467),5,main]
java.lang.OutOfMemoryError: Java heap space


Py4JJavaError: An error occurred while calling o98.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 25.0 failed 1 times, most recent failure: Lost task 5.0 in stage 25.0 (TID 464) (192.168.0.15 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2098)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2228)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1687)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:489)
	at java.base/java.io.ObjectInputStream.readObject(ObjectIn

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob

# Arrêter la session Spark existante s'il y en a une
if SparkSession.builder.getOrCreate().sparkContext:
    SparkSession.builder.getOrCreate().sparkContext.stop()

# Initialiser une nouvelle session Spark
spark = SparkSession.builder \
    .appName("PCA Example") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.maxResultSize", "8g") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "16g") \
    .getOrCreate()

# Chemin vers le dossier contenant les images
image_folder = './data/Test1/Apple Braeburn/'
image_files = glob.glob(image_folder + '*.jpg')

# Fonction pour charger et convertir une image en un vecteur
def load_image_as_vector(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image_resized = cv2.resize(image, (64, 64))  # Redimensionner pour la simplicité
    image_vector = image_resized.flatten().tolist()  # Aplatir en un seul vecteur
    return (Vectors.dense(image_vector),)

# Charger les images et créer un DataFrame Spark
image_vectors = [load_image_as_vector(image_file) for image_file in image_files]
df = spark.createDataFrame(image_vectors, ["features"])

# Nettoyer les données
df = df.na.drop()  # Supprimer les lignes avec des valeurs nulles
df = df.repartition(100)  # Répartir les données en 100 partitions

# Afficher quelques lignes pour vérifier le DataFrame
df.show(5)

# Appliquer PCA pour la réduction de dimension
pca = PCA(k=20, inputCol="features", outputCol="pca_features")

try:
    pca_model = pca.fit(df)
    df_pca = pca_model.transform(df)
    print("PCA appliqué avec succès.")
except Exception as e:
    print(f"Erreur lors de l'application du PCA: {e}")

# Récupérer les composantes principales
if 'pca_model' in locals():
    components = np.array(pca_model.pc.toArray())

    # Fonction pour inverser PCA
    def inverse_pca(pca_features, components, mean_vector):
        return np.dot(pca_features, components.T) + mean_vector

    # Sélectionner une image et ses caractéristiques PCA
    sample_image_pca = df_pca.select("pca_features").first()["pca_features"]
    mean_vector = pca_model.mean.toArray()

    # Reconstituer l'image
    reconstructed_image_vector = inverse_pca(sample_image_pca, components, mean_vector)
    reconstructed_image = np.array(reconstructed_image_vector).reshape((64, 64))

    # Afficher l'image reconstruite
    #plt.imshow(reconstructed_image, cmap='gray')
    #plt.show()
else:
    print("Le modèle PCA n'a pas été créé.")


Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true
Picked up _JAVA_OPTIONS: -Dawt.useSystemAAFontSettings=on -Dswing.aatext=true
24/05/27 21:45:44 WARN Utils: Your hostname, KALIDESKTOP resolves to a loopback address: 127.0.1.1; using 192.168.0.20 instead (on interface wlan0)
24/05/27 21:45:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/27 21:45:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/27 21:45:55 WARN TaskSetManager: Stage 0 contains a task of very large size (4438 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+
|            features|
+--------------------+
|[255.0,254.0,254....|
|[254.0,254.0,254....|
|[254.0,254.0,255....|
|[255.0,255.0,255....|
|[255.0,255.0,255....|
+--------------------+
only showing top 5 rows



24/05/27 21:46:00 WARN TaskSetManager: Stage 5 contains a task of very large size (4438 KiB). The maximum recommended task size is 1000 KiB.
24/05/27 21:46:07 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/05/27 21:46:44 ERROR Executor: Exception in task 2.0 in stage 17.0 (TID 225)]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:64)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:363)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1$adapted(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$$$Lambda$2676/0x00007f87e0cc1a88.apply(Unknown Source)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.

Erreur lors de l'application du PCA: An error occurred while calling o89.fit.
: org.apache.spark.SparkException: Job 8 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1(DAGScheduler.scala:1253)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1$adapted(DAGScheduler.scala:1251)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:1251)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:3087)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$stop$3(DAGScheduler.scala:2973)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1375)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2973)
	at org.apache.spark.SparkContext.$anonfun$stop$12(SparkContext.scala:2263)

24/05/27 21:46:45 ERROR TaskSchedulerImpl: Exception in statusUpdate
java.util.concurrent.RejectedExecutionException: Task org.apache.spark.scheduler.TaskResultGetter$$anon$3@aafdb24 rejected from java.util.concurrent.ThreadPoolExecutor@15a6f5c4[Shutting down, pool size = 2, active threads = 2, queued tasks = 0, completed tasks = 224]
	at java.base/java.util.concurrent.ThreadPoolExecutor$AbortPolicy.rejectedExecution(ThreadPoolExecutor.java:2065)
	at java.base/java.util.concurrent.ThreadPoolExecutor.reject(ThreadPoolExecutor.java:833)
	at java.base/java.util.concurrent.ThreadPoolExecutor.execute(ThreadPoolExecutor.java:1365)
	at org.apache.spark.scheduler.TaskResultGetter.enqueueSuccessfulTask(TaskResultGetter.scala:61)
	at org.apache.spark.scheduler.TaskSchedulerImpl.liftedTree2$1(TaskSchedulerImpl.scala:836)
	at org.apache.spark.scheduler.TaskSchedulerImpl.statusUpdate(TaskSchedulerImpl.scala:811)
	at org.apache.spark.scheduler.local.LocalEndpoint$$anonfun$receive$1.applyOrElse(Local

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 47190)
Traceback (most recent call last):
  File "/home/anonyme2/anaconda3/envs/cloud-computing/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/anonyme2/anaconda3/envs/cloud-computing/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/anonyme2/anaconda3/envs/cloud-computing/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/home/anonyme2/anaconda3/envs/cloud-computing/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/anonyme2/anaconda3/envs/cloud-computing/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/home/anonyme2/anaconda3/envs/cloud-computing/li

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob

# Arrêter la session Spark existante s'il y en a une
if SparkSession.builder.getOrCreate().sparkContext:
    SparkSession.builder.getOrCreate().sparkContext.stop()

# Initialiser une nouvelle session Spark
spark = SparkSession.builder \
    .appName("PCA Example") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.maxResultSize", "8g") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "16g") \
    .getOrCreate()

# Chemin vers le dossier contenant les images
image_folder = './data/Test2/Apple Braeburn/'
image_files = glob.glob(image_folder + '*.jpg')

# Fonction pour charger et convertir une image en un vecteur
def load_image_as_vector(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image_resized = cv2.resize(image, (64, 64))  # Redimensionner pour la simplicité
    image_vector = image_resized.flatten().tolist()  # Aplatir en un seul vecteur
    return (Vectors.dense(image_vector),)

# Charger les images et créer un DataFrame Spark
image_vectors = [load_image_as_vector(image_file) for image_file in image_files]
df = spark.createDataFrame(image_vectors, ["features"])

# Nettoyer les données
df = df.na.drop()  # Supprimer les lignes avec des valeurs nulles
df = df.repartition(10)  # Répartir les données en 100 partitions

# Afficher quelques lignes pour vérifier le DataFrame
df.show(5)

pca = PCA(k=20, inputCol="features", outputCol="pca_features")

try:
    pca_model = pca.fit(df)
    df_pca = pca_model.transform(df)
    print("PCA appliqué avec succès.")
except Exception as e:
    print(f"Erreur lors de l'application du PCA: {e}")

# Récupérer les composantes principales
if 'pca_model' in locals():
    components = np.array(pca_model.pc.toArray())

    # Fonction pour inverser PCA
    def inverse_pca(pca_features, components, mean_vector):
        return np.dot(pca_features, components.T) + mean_vector

    # Sélectionner une image et ses caractéristiques PCA
    sample_image_pca = df_pca.select("pca_features").first()["pca_features"]
    mean_vector = pca_model.mean.toArray()

    # Reconstituer l'image
    reconstructed_image_vector = inverse_pca(sample_image_pca, components, mean_vector)
    reconstructed_image = np.array(reconstructed_image_vector).reshape((64, 64))

    # Afficher l'image reconstruite
    #plt.imshow(reconstructed_image, cmap='gray')
    #plt.show()
else:
    print("Le modèle PCA n'a pas été créé.")


+--------------------+
|            features|
+--------------------+
|[255.0,255.0,255....|
|[255.0,254.0,254....|
|[255.0,255.0,255....|
|[254.0,255.0,255....|
|[254.0,254.0,255....|
+--------------------+
only showing top 5 rows



24/05/27 22:05:28 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Erreur lors de l'application du PCA: An error occurred while calling o223.fit.
: java.lang.OutOfMemoryError: Java heap space
	at breeze.linalg.svd$.breeze$linalg$svd$$doSVD_Double(svd.scala:94)
	at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:36)
	at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:35)
	at breeze.generic.UFunc.apply(UFunc.scala:47)
	at breeze.generic.UFunc.apply$(UFunc.scala:46)
	at breeze.linalg.svd$.apply(svd.scala:21)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.computePrincipalComponentsAndExplainedVariance(RowMatrix.scala:501)
	at org.apache.spark.mllib.feature.PCA.fit(PCA.scala:65)
	at org.apache.spark.ml.feature.PCA.fit(PCA.scala:93)
	at org.apache.spark.ml.feature.PCA.fit(PCA.scala:64)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMeth